In [1]:
import requests
import pandas as pd

In [2]:
teams = ['crd',
         'atl',
         'rav',
         'buf',
         'car',
         'chi',
         'cin',
         'cle',
         'dal',
         'den',
         'det',
         'gnb',
         'htx',
         'clt',
         'jax',
         'kan',
         'sdg',
         'ram',
         'mia',
         'min',
         'nwe',
         'nor',
         'nyg',
         'nyj',
         'rai',
         'phi',
         'pit',
         'sfo',
         'sea',
         'tam',
         'oti',
         'was'
        ]
len(teams)

32

In [3]:
year_range = range(2002,2020)
data_dict = {}

In [4]:
%%time
for team in teams:
    print(team)
    for year in year_range:
        print(f'{team} - {year}')
        url = f'https://www.pro-football-reference.com/teams/{team}/{year}.htm'
        data = pd.read_html(url, header=1)
        data_dict[year] = data[1]
        data_dict[year]['year'] = year
        data_dict[year]['team'] = team

crd
crd - 2002
crd - 2003
crd - 2004
crd - 2005
crd - 2006
crd - 2007
crd - 2008
crd - 2009
crd - 2010
crd - 2011
crd - 2012
crd - 2013
crd - 2014
crd - 2015
crd - 2016
crd - 2017
crd - 2018
crd - 2019
atl
atl - 2002
atl - 2003
atl - 2004
atl - 2005
atl - 2006
atl - 2007
atl - 2008
atl - 2009
atl - 2010
atl - 2011
atl - 2012
atl - 2013
atl - 2014
atl - 2015
atl - 2016
atl - 2017
atl - 2018
atl - 2019
rav
rav - 2002
rav - 2003
rav - 2004
rav - 2005
rav - 2006
rav - 2007
rav - 2008
rav - 2009
rav - 2010
rav - 2011
rav - 2012
rav - 2013
rav - 2014
rav - 2015
rav - 2016
rav - 2017
rav - 2018
rav - 2019
buf
buf - 2002
buf - 2003
buf - 2004
buf - 2005
buf - 2006
buf - 2007
buf - 2008
buf - 2009
buf - 2010
buf - 2011
buf - 2012
buf - 2013
buf - 2014
buf - 2015
buf - 2016
buf - 2017
buf - 2018
buf - 2019
car
car - 2002
car - 2003
car - 2004
car - 2005
car - 2006
car - 2007
car - 2008
car - 2009
car - 2010
car - 2011
car - 2012
car - 2013
car - 2014
car - 2015
car - 2016
car - 2017
car - 2018
c

In [5]:
data_dict

{2002:     Week  Day          Date Unnamed: 3 Unnamed: 4 Unnamed: 5  OT  Rec  \
 0      1  Sun   September 8  1:05PM ET   boxscore          W NaN  1-0   
 1      2  Mon  September 16  9:05PM ET   boxscore          L NaN  1-1   
 2      3  Sun  September 22  4:15PM ET   boxscore          L NaN  1-2   
 3      4  NaN           NaN        NaN        NaN        NaN NaN  NaN   
 4      5  Sun     October 6  1:03PM ET   boxscore          W NaN  2-2   
 5      6  Sun    October 13  1:01PM ET   boxscore          L NaN  2-3   
 6      7  Sun    October 20  4:15PM ET   boxscore          L NaN  2-4   
 7      8  Sun    October 27  8:35PM ET   boxscore          W NaN  3-4   
 8      9  Sun    November 3  4:15PM ET   boxscore          W NaN  4-4   
 9     10  Sun   November 10  4:05PM ET   boxscore          L NaN  4-5   
 10    11  Sun   November 17  1:04PM ET   boxscore          L NaN  4-6   
 11    12  Sun   November 24  1:01PM ET   boxscore          W NaN  5-6   
 12    13  Thu   November 28  4: